# Building term-specific datasets
## aggregated by (case, justice)

The goal here is to construct datasets specific to a term that have 700-1400 rows. Each row will represent everything said on a particular case by a particular justice. Empty rows will be discarded.

In [1]:
import pandas as pd


In [3]:
ts = pd.read_csv('/Volumes/TARDIS/oralarg/ts_double_quotes.csv', encoding = 'latin-1')
ts.head()

/Users/tunder/miniconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (1,33,34,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,95,96,97,98,99) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CaseID,caseId,audioRef,argRef,term,caseName,argType,rearg,oaDate,section,...,interAJMM,interJJMF,interJAMF,interAJMF,interJJFM,interJAFM,interAJFM,interJJFF,interJAFF,interAJFF
0,59177.0,1955-049,13100,13092,1955,Pennsylvania v. Nelson,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,59177.0,1955-049,13092,13092,1955,Pennsylvania v. Nelson,original (1/1),0.0,15nov1955,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,59177.0,1955-049,13092,13092,1955,Pennsylvania v. Nelson,original (1/1),0.0,15nov1955,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,59177.0,1955-049,13092,13092,1955,Pennsylvania v. Nelson,original (1/1),0.0,15nov1955,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,59177.0,1955-049,13092,13092,1955,Pennsylvania v. Nelson,original (1/1),0.0,15nov1955,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
terms = set(ts.term)

In [28]:
for t in terms:
    df = ts.loc[ts.term == t, : ]
    speakercases = []
    speakers = []
    texts = []
    wordlens = []
    mqscore = []
    
    grouped = df.groupby(['caseId', 'justiceName'])
    
    for idx, group in grouped:
        if pd.isnull(idx[1]):
            continue
        
        text = ' '.join([x for x in group.fulltext if not pd.isnull(x)])
        words = len(text.split())
        if words < 1:
            continue
        
        meanmq = np.mean(group.mqscore)
        mqstd = np.std(group.mqscore)
        if mqstd > 0.001:
            print('error', t)
        mqscore.append(meanmq)
            
        speakeridx = '-'.join(idx)
        speakercases.append(speakeridx)
        speakers.append(idx[1])
        
        texts.append(text)
        
        wordlens.append(words)
    
    termdf = pd.DataFrame({'speakercase': speakercases, "justice": speakers, 'text': texts, 'wordct': wordlens, 'mq': mqscore})
    filename = '/Volumes/TARDIS/oralarg/termgrouped/term' + str(t) + '.tsv'
    termdf.to_csv(filename, sep = '\t', index = False)
    
    
        
        
        